In [1]:
!pip install -q sentence-transformers hnswlib timm torchvision lightgbm xgboost tqdm

import os
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
import torch
import torchvision.transforms as T
import timm
import hnswlib
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import lightgbm as lgb
import joblib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 31.5 MB/

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [3]:
DATA_ROOT = '/kaggle/input/amazonmlc-testdata'  
train_path = os.path.join(DATA_ROOT, 'train.csv')
test_path  = os.path.join(DATA_ROOT, 'test.csv')
image_folder = os.path.join(DATA_ROOT, 'images') 

# writable cache & outputs on Kaggle
cache_folder = '/kaggle/working/image_embeddings_cache'
os.makedirs(cache_folder, exist_ok=True)
os.makedirs('/kaggle/working/models', exist_ok=True)

# load
print("Listing /kaggle/input contents:")
for d,_,files in os.walk('/kaggle/input'):
    print(d, len(files))

Listing /kaggle/input contents:
/kaggle/input 0
/kaggle/input/amazonmlc-testdata 2


In [4]:
train = pd.read_csv(train_path, sep=',', quotechar='"', engine='python', on_bad_lines='skip')
test  = pd.read_csv(test_path,  sep=',', quotechar='"', engine='python', on_bad_lines='skip')
print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (75000, 4)
Test shape: (75000, 3)


In [5]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower().replace('\n',' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

train['clean_text'] = train['catalog_content'].apply(clean_text)
test['clean_text'] = test['catalog_content'].apply(clean_text)

In [6]:
# Extract numeric value and unit
def extract_value_unit(text):
    value, unit = None, None
    value_match = re.search(r'Value[: ]+\s*([\d\.]+)', text, re.IGNORECASE)
    if value_match:
        try: value = float(value_match.group(1))
        except: value = None
    unit_match = re.search(r'Unit[: ]+\s*([a-zA-Z]+)', text, re.IGNORECASE)
    if unit_match: unit = unit_match.group(1).lower()
    return value, unit

train[['value_num', 'unit']] = train['catalog_content'].apply(lambda x: pd.Series(extract_value_unit(x)))
test[['value_num', 'unit']] = test['catalog_content'].apply(lambda x: pd.Series(extract_value_unit(x)))

In [7]:
# Extract brand
def extract_brand(text):
    if pd.isna(text): return None
    brand_match = re.match(r'([^,]+)', text)
    if brand_match: return brand_match.group(1).strip().lower()
    return None

train['brand'] = train['catalog_content'].apply(extract_brand)
test['brand'] = test['catalog_content'].apply(extract_brand)

In [8]:
#Categorical features
TOP_K_BRANDS = 50
top_brands = train['brand'].value_counts().nlargest(TOP_K_BRANDS).index.tolist()
train_top_brands = train['brand'].apply(lambda x: x if x in top_brands else 'other')
test_top_brands = test['brand'].apply(lambda x: x if x in top_brands else 'other')

train_brands_onehot = pd.get_dummies(train_top_brands, prefix='brand', dummy_na=True)
test_brands_onehot = pd.get_dummies(test_top_brands, prefix='brand', dummy_na=True)
test_brands_onehot = test_brands_onehot.reindex(columns=train_brands_onehot.columns, fill_value=0)

In [9]:
# Target mean encoding with blending for rare
brand_target_mean = train.groupby('brand')['price'].mean().to_dict()
global_mean = train['price'].mean()
alpha = 0.7
train['brand_target'] = train['brand'].apply(lambda x: alpha*brand_target_mean.get(x, global_mean) + (1-alpha)*global_mean)
test['brand_target']  = test['brand'].apply(lambda x: alpha*brand_target_mean.get(x, global_mean) + (1-alpha)*global_mean)

In [10]:
# Units
TOP_K_UNITS = 20
top_units = train['unit'].value_counts().nlargest(TOP_K_UNITS).index.tolist()
train_top_units = train['unit'].apply(lambda x: x if x in top_units else 'other')
test_top_units = test['unit'].apply(lambda x: x if x in top_units else 'other')

train_units_onehot = pd.get_dummies(train_top_units, prefix='unit', dummy_na=True)
test_units_onehot = pd.get_dummies(test_top_units, prefix='unit', dummy_na=True)
test_units_onehot = test_units_onehot.reindex(columns=train_units_onehot.columns, fill_value=0)

unit_target_mean = train.groupby('unit')['price'].mean().to_dict()
train['unit_target'] = train['unit'].apply(lambda x: alpha*unit_target_mean.get(x, global_mean) + (1-alpha)*global_mean)
test['unit_target'] = test['unit'].apply(lambda x: alpha*unit_target_mean.get(x, global_mean) + (1-alpha)*global_mean)

In [11]:
# Structured arrays
X_train_cat = np.hstack([train_brands_onehot.values, train['brand_target'].values.reshape(-1,1),
                         train_units_onehot.values, train['unit_target'].values.reshape(-1,1)]).astype(np.float32)
X_test_cat = np.hstack([test_brands_onehot.values, test['brand_target'].values.reshape(-1,1),
                        test_units_onehot.values, test['unit_target'].values.reshape(-1,1)]).astype(np.float32)

X_train_num = train[['value_num']].fillna(0).astype(np.float32).values
X_test_num = test[['value_num']].fillna(0).astype(np.float32).values

In [12]:
model_names = [
    'sentence-transformers/paraphrase-MiniLM-L6-v2',
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/distiluse-base-multilingual-cased-v2'
]

X_train_text_list, X_test_text_list = [], []

for mname in model_names:
    print(f"Encoding with {mname}...")

    model = SentenceTransformer(mname, device=device)

    # Encode train and test separately
    emb_train = model.encode(
        train['clean_text'].tolist(),
        convert_to_tensor=False,
        batch_size=32,  # smaller batch for Kaggle
        show_progress_bar=True
    )
    X_train_text_list.append(emb_train)

    emb_test = model.encode(
        test['clean_text'].tolist(),
        convert_to_tensor=False,
        batch_size=32,  # smaller batch for Kaggle
        show_progress_bar=True
    )
    X_test_text_list.append(emb_test)

    # Free GPU memory
    del model
    torch.cuda.empty_cache()

X_train_text = np.hstack(X_train_text_list)
X_test_text = np.hstack(X_test_text_list)

print("Text embeddings shape:", X_train_text.shape)


Encoding with sentence-transformers/paraphrase-MiniLM-L6-v2...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Batches:   0%|          | 0/2344 [00:00<?, ?it/s]

Batches:   0%|          | 0/2344 [00:00<?, ?it/s]

Text embeddings shape: (75000, 1280)


In [13]:
def generate_ann_features(train_emb, test_emb, k=10):
    dim = train_emb.shape[1]
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=train_emb.shape[0], ef_construction=200, M=50)
    index.add_items(train_emb)
    index.set_ef(50)
    # Train features
    labels_train, _ = index.knn_query(train_emb, k=k)
    ann_train = np.array([train['price'].values[labels_train[i]] for i in range(len(labels_train))])
    # Test features
    labels_test, _ = index.knn_query(test_emb, k=k)
    ann_test = np.array([train['price'].values[labels_test[i]] for i in range(len(labels_test))])
    return ann_train, ann_test

from sklearn.model_selection import KFold

def generate_ann_features_cv(embeddings, targets, test_emb, n_splits=5, k=10):
    dim = embeddings.shape[1]
    ann_train = np.zeros((embeddings.shape[0], k))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for tr_idx, val_idx in kf.split(embeddings):
        index = hnswlib.Index(space='cosine', dim=dim)
        index.init_index(max_elements=len(tr_idx), ef_construction=200, M=50)
        index.add_items(embeddings[tr_idx])
        index.set_ef(50)
        labels, _ = index.knn_query(embeddings[val_idx], k=k)
        ann_train[val_idx] = np.array([targets[tr_idx][idxs] for idxs in labels])

    # build one final ANN for test using all train embeddings
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=len(embeddings), ef_construction=200, M=50)
    index.add_items(embeddings)
    index.set_ef(50)
    labels_test, _ = index.knn_query(test_emb, k=k)
    ann_test = np.array([targets[idxs] for idxs in labels_test])

    return ann_train, ann_test

ANN_train, ANN_test = generate_ann_features_cv(X_train_text, train['price'].values, X_test_text, n_splits=5, k=10)
print("ANN features shape:", ANN_train.shape)

ANN features shape: (75000, 10)


In [14]:
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

In [15]:
def batch_image_embeddings(image_names, backbones=['resnet50'], batch_size=32):
    all_embeddings = []
    for backbone_name in backbones:
        print(f"Using backbone: {backbone_name}")
        if backbone_name.startswith('resnet'):
            model = timm.create_model(backbone_name, pretrained=True, num_classes=0, global_pool='avg').to(device)
        else:
            model = timm.create_model(backbone_name, pretrained=True, num_classes=0, global_pool='avg').to(device)
        model.eval()
        emb_list = []
        for i in tqdm(range(0, len(image_names), batch_size)):
            batch_files = image_names[i:i+batch_size]
            imgs, indices = [], []
            for idx, img_name in enumerate(batch_files):
                safe_name = os.path.splitext(os.path.basename(img_name))[0]
                cache_path = os.path.join(cache_folder, f"{backbone_name}_{safe_name}.npy")
                if os.path.exists(cache_path):
                    emb_list.append(np.load(cache_path))
                else:
                    img_path = os.path.join(image_folder, img_name)
                    try:
                        img = Image.open(img_path).convert("RGB")
                        imgs.append(transform(img))
                        indices.append(len(emb_list))
                        emb_list.append(None)
                    except:
                        emb_list.append(np.zeros(model.num_features))
            if len(imgs) > 0:
                imgs = torch.stack(imgs).to(device)
                with torch.no_grad():
                    batch_emb = model(imgs).cpu().numpy()
                for idx, emb in zip(indices, batch_emb):
                    emb_list[idx] = emb
                    safe_name = os.path.splitext(os.path.basename(batch_files[indices.index(idx)]))[0]
                    cache_path = os.path.join(cache_folder, f"{backbone_name}_{safe_name}.npy")
                    np.save(cache_path, emb)
        all_embeddings.append(np.vstack(emb_list))
    return np.hstack(all_embeddings)

X_train_img = batch_image_embeddings(train['image_link'].tolist(), backbones=['resnet50','efficientnet_b0'], batch_size=32)
X_test_img  = batch_image_embeddings(test['image_link'].tolist(), backbones=['resnet50','efficientnet_b0'], batch_size=32)
print("Image embeddings shape:", X_train_img.shape)

Using backbone: resnet50


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

  0%|          | 0/2344 [00:00<?, ?it/s]

Using backbone: efficientnet_b0


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

...

Using backbone: resnet50


  0%|          | 0/2344 [00:00<?, ?it/s]

Using backbone: efficientnet_b0


  0%|          | 0/2344 [00:00<?, ?it/s]

Image embeddings shape: (75000, 3328)


In [16]:
X_train_full = np.hstack([X_train_text, ANN_train, X_train_num, X_train_cat, X_train_img])
X_test_full  = np.hstack([X_test_text, ANN_test, X_test_num, X_test_cat, X_test_img])
y_train = np.log1p(train['price'].values)
print("Full feature matrix shape:", X_train_full.shape)

Full feature matrix shape: (75000, 4695)


In [17]:
# Train-validation split
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_train_full))
test_preds = np.zeros(len(X_test_full))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\n===== Fold {fold+1} =====")
    X_tr, X_val = X_train_full[tr_idx], X_train_full[val_idx]
    y_tr, y_val = y_train[tr_idx], y_train[val_idx]

    xgb = XGBRegressor(
        n_estimators=2000, max_depth=9, learning_rate=0.03,
        subsample=0.9, colsample_bytree=0.9, tree_method='gpu_hist',
        reg_lambda=1.0, reg_alpha=0.3, random_state=fold
    )
    xgb.fit(X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=100,
            verbose=False)
    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_val = lgb.Dataset(X_val, y_val)
    lgbm = lgb.train(
        lgb_params,
        lgb_train,
        num_boost_round=2000,
        valid_sets=[lgb_val],
        callbacks=[early_stopping(stopping_rounds=100)]
    )
    pred_val = 0.5*(xgb.predict(X_val) + lgbm.predict(X_val))
    oof_preds[val_idx] = np.expm1(pred_val)
    test_preds += 0.5*(np.expm1(xgb.predict(X_test_full)) + np.expm1(lgbm.predict(X_test_full)))/kf.n_splits
        # Overfitting diagnostics
    y_tr_pred_xgb = np.expm1(xgb.predict(X_tr))
    y_val_pred_xgb = np.expm1(xgb.predict(X_val))

    y_tr_pred_lgb = np.expm1(lgbm.predict(X_tr))
    y_val_pred_lgb = np.expm1(lgbm.predict(X_val))

    y_tr_pred = 0.5 * (y_tr_pred_xgb + y_tr_pred_lgb)
    y_val_pred = 0.5 * (y_val_pred_xgb + y_val_pred_lgb)

    y_tr_true = np.expm1(y_tr)
    y_val_true = np.expm1(y_val)

    train_rmse = mean_squared_error(y_tr_true, y_tr_pred, squared=False)
    val_rmse = mean_squared_error(y_val_true, y_val_pred, squared=False)

    print(f"\nFold {fold+1} Overfitting Diagnostic")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Validation RMSE: {val_rmse:.4f}")
    print(f"Gap: {val_rmse - train_rmse:.4f}")




===== Fold 1 =====


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:27:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


NameError: name 'lgb_params' is not defined

In [ ]:
val_smape = smape(np.expm1(y_train), oof_preds)
print(f"Overall 5-Fold SMAPE: {val_smape:.2f}%")
submission = pd.DataFrame({'sample_id': test['sample_id'], 'price': test_preds})
submission.to_csv('/kaggle/working/submission_5fold.csv', index=False)

In [ ]:
xgb_model = XGBRegressor(
    n_estimators=2000,
    max_depth=9,
    learning_rate=0.03,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,        # mild L2
    reg_alpha=0.3,         # mild L1
    tree_method='gpu_hist',
    random_state=42
)

In [ ]:
from lightgbm import early_stopping, log_evaluation

lgb_train = lgb.Dataset(X_tr, y_tr)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

lgb_params = {
    'objective':'regression',
    'metric':'rmse',
    'boosting_type':'gbdt',
    'learning_rate':0.03,
    'num_leaves': 512,
    'max_depth': -1,
    'feature_fraction':0.9,
    'bagging_fraction':0.9,
    'bagging_freq':1,
    'lambda_l1':0.2,
    'lambda_l2':0.8,
    'min_data_in_leaf':30,
    'device':'gpu',
    'seed':42
}

lgb_model = lgb.train(
    lgb_params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_val],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=50)
    ]
)

In [ ]:
# Ensemble Predictions
def smape(y_true, y_pred):
    return 100*np.mean(np.abs(y_pred - y_true)/((np.abs(y_true)+np.abs(y_pred))/2))

# y_val_pred = 0.5*(np.expm1(xgb_model.predict(X_val)) + np.expm1(lgb_model.predict(X_val)))
# val_smape = smape(np.expm1(y_val), y_val_pred)
# print(f"Validation SMAPE: {val_smape:.2f}%")
val_smape = smape(np.expm1(y_train), oof_preds)
print(f"Overall 5-Fold SMAPE: {val_smape:.2f}%")

In [ ]:
# Train final models on all training data after CV
xgb_model.fit(X_train_full, y_train)

lgb_train = lgb.Dataset(X_train_full, y_train)
lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=1000)  # you can adjust rounds

In [ ]:
y_test_pred = 0.5*(np.expm1(xgb_model.predict(X_test_full)) + np.expm1(lgb_model.predict(X_test_full)))
submission = pd.DataFrame({'sample_id': test['sample_id'], 'price': y_test_pred})
submission_path = '/kaggle/working/test_out.csv'
submission.to_csv(submission_path, index=False)
print("Submission saved to:", submission_path)

# Save models to working (will appear in Outputs)
joblib.dump(xgb_model, '/kaggle/working/models/xgb_model_embed.pkl')
joblib.dump(lgb_model, '/kaggle/working/models/lgb_model_embed.pkl')
print("Models saved to /kaggle/working/models/")